In [120]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import seaborn as sb
%pylab inline
import calendar
import datetime as dt

from pylab import rcParams
rcParams['figure.figsize'] = 20, 14

import fnmatch
import os
from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


# Читаем и преобразовываем JSON

In [121]:
print('Обработка файлов JSON')

firstdata=True
rootDir='json_data'
files=os.listdir(rootDir)
pbar = tqdm(total=len(files))
datas=[]
for file in files:
    pbar.update(1)
    if fnmatch.fnmatch(file, '*.json'):
        with open(rootDir+'/'+file, encoding='utf8') as data_file:
            json_data = json.load(data_file)
            if firstdata:
                _columns=[val['Name']['#value'] for val in json_data['#value']['column']]#Вычленяем колонки
                firstdata=True
            datas.append(pd.DataFrame(data=json_data['#value']['row'], columns=_columns).applymap(lambda cell: cell['#value']))#читаем JSON и вычленяем значения
pbar.close()

print('Преобразуем в датафрейм...')

data=pd.concat(datas, axis=0, join='outer', ignore_index=True)

del json_data            

print('Готово')

Обработка файлов JSON


100%|██████████| 1216/1216 [00:36<00:00, 33.52it/s]


Преобразуем в датафрейм...
Готово


# Преобразование характеристик

In [122]:
data['Дата']=pd.to_datetime(data['Дата'],format='%Y-%m-%d')#преобразуем дату

data['ТоварСреднийВес']=data['ТоварСреднийВес'].map(lambda cell: cell if np.isreal(cell) else 0).astype(float)#заполняем пустоты

# Формируем новые характеристики

In [123]:
#параметры даты
data['Год']=data['Дата'].dt.year
data['Месяц']=data['Дата'].dt.month
data['Неделя']=data['Дата'].dt.week
data['День']=data['Дата'].dt.day
data['ДеньНедели']=data['Дата'].dt.weekday
#Понедельник-четверг 0..3, пятница - 4, суббота-воскресенье - 5..6
data['РабочиеВыходные']=data['ДеньНедели'].map(lambda cell: 1 if cell<4 else 2 if cell==4 else 3)

data['НеЗаказыватьУПоставщиков']=data['НеЗаказыватьУПоставщиков'].astype('int')

# Праздники

In [124]:
with open('Праздники 20140101 - 20170531.json', encoding='utf8') as data_file:
    json_data = json.load(data_file)
_col=[val['Name']['#value'] for val in json_data['#value']['column']]
celebrate=pd.DataFrame(data=json_data['#value']['row'], columns=_col).applymap(lambda cell: cell['#value'])
celebrate['Праздник']=pd.to_datetime(celebrate['Праздник'],format='%Y-%m-%d')#преобразуем дату

del json_data

data['Праздник']=0#Обычный день
data.loc[data['Дата'].isin(celebrate['Праздник']),'Праздник']=1#праздник


del celebrate

# Удаляем ненужное

In [125]:
data.drop(['Дата','Товар'], axis=1, inplace=True)

# 29 февраля: берем среднее за 28 и 29 февраля

In [126]:
condition29=(data['День'] > 28)&(data['Месяц']==2)#выборка 29 февраля каждого года
for year in data.loc[condition29,'Год'].unique():#Какие года високосные
    condition28=(data['День'] == 28)&(data['Месяц']==2)&(data['Год']==year)#выборка 28 февраля каждого года
    row=data[condition28].iloc[0]#первая строка колонки
    for col in['День','Неделя','ДеньНедели','РабочиеВыходные','Праздник']:#приравниваем характеристики даты
        data.loc[condition29,col]=row[col]

#группируемые колонки
selectColumns=list(set(data.columns)-set(['Количество']))

#среднее по 28 февраля
condition28=(data['День'] == 28)&(data['Месяц']==2)
feb28=data[condition28].groupby(selectColumns)['Количество'].mean().reset_index()

#удаляем из оригинально датасета все 28 февраля
data.drop(data[condition28].index, inplace=True)

#добавляем среднее за 28 февраля
data=pd.concat([data,feb28], axis=0, join='outer', ignore_index=True)

## Группировка данных с суммированием количества

In [127]:
#группируемые колонки
selectColumns=list(set(data.columns)-set(['Количество']))

data=data.groupby(selectColumns)['Количество'].sum().reset_index()

# Сохраняем в формате Pickle (Python)

In [128]:
data.to_pickle('MG_Sales.pickle',compression='gzip')

In [129]:
print('Завершено')

Завершено
